### 라이브러리 설치

### 클래스 별로 이미지 정리

In [1]:
import os
import pandas as pd
import shutil
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('./data/train.csv')
train_df, valid_df = train_test_split(df, test_size=0.1, random_state=42)
train_df.reset_index(inplace=True, drop=True)
valid_df.reset_index(inplace=True, drop=True)

train_df.to_csv("./data/train_df.csv", index=False)
valid_df.to_csv("./data/valid_df.csv", index=False)

In [3]:
train_dir = "./prepare_data/train/"
valid_dir = "./prepare_data/valid/"

def make_data(df, output_dir):
    for index, row in tqdm(df.iterrows()):
        src_path = os.path.join('./data', row['img_path'].lstrip('./'))
        label = row['label']
        dest_dir = os.path.join(output_dir, label)
    
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)
    
        file_name = os.path.basename(src_path)
        dest_path = os.path.join(dest_dir, file_name)

        # 파일 존재 여부 확인
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_path)
        else:
            print(f"파일이 존재하지 않습니다: {src_path}")
            
    print("파일 이동이 완료되었습니다.")

In [4]:
make_data(train_df, train_dir)
make_data(valid_df, valid_dir)

0it [00:00, ?it/s]

파일 이동이 완료되었습니다.


0it [00:00, ?it/s]

파일 이동이 완료되었습니다.


### Huggingface Library를 이용한 이미지 분류

In [5]:
import warnings
warnings.filterwarnings('ignore')
import random
import torch
import numpy as np
import os
from datasets import load_dataset
from evaluate import load
from transformers import AutoModelForImageClassification, AutoImageProcessor, TrainingArguments, Trainer

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
dataset = load_dataset("imagefolder", data_dir="./prepare_data/")
dataset = dataset.rename_column("label", "labels")

model_name = "microsoft/swinv2-large-patch4-window12to16-192to256-22kto1k-ft"
processor = AutoImageProcessor.from_pretrained(model_name)

Resolving data files:   0%|          | 0/14250 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1584 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [7]:
def transform(example_batch):
    inputs = processor([x for x in example_batch['image']], return_tensors='pt')
    inputs['labels'] = example_batch['labels']
    return inputs

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [8]:
prepared_ds = dataset.with_transform(transform).shuffle()

metric = load("accuracy", trust_remote_code=True)
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

In [9]:
labels = dataset['train'].features['labels'].names

model = AutoModelForImageClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes=True
)

Some weights of Swinv2ForImageClassification were not initialized from the model checkpoint at microsoft/swinv2-large-patch4-window12to16-192to256-22kto1k-ft and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1536]) in the checkpoint and torch.Size([25, 1536]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([25]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
training_args = TrainingArguments(
    output_dir="./results/swinvit-experience-1",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=2,
    num_train_epochs=200,
    learning_rate=5e-5,
    remove_unused_columns=False,
    label_smoothing_factor=0.1,
    warmup_ratio=0.1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    seed=42
)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=processor,
)

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

In [ ]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

# 더 작게 돌리기 안돌아감

### 추론

In [ ]:
import os
import pandas as pd
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification, pipeline
from tqdm.auto import notebook_tqdm

In [ ]:
model_name = "microsoft/swinv2-large-patch4-window12to16-192to256-22kto1k-ft"
processor = AutoImageProcessor.from_pretrained(model_name, return_tensor="pt")

In [ ]:
model = AutoModelForImageClassification.from_pretrained("./best/checkpoint-2341/")
model.eval()

classifier = pipeline("image-classification", model=model, image_processor=processor)

In [ ]:
test_df = pd.read_csv("./data/test.csv")
test_df.head()

In [ ]:
test_file_name = test_df['img_path'].to_list()

predict_labels = []

for idx, name in notebook_tqdm(enumerate(test_file_name), total=len(test_file_name)):
    image = Image.open(os.path.join("./data/", name.strip('./')))
    predict_labels.append(classifier(image)[0]['label'])

In [ ]:
submission_df = pd.read_csv("./data/sample_submission.csv")
submission_df.head()
submission_df['label'] = predict_labels
submission_df.to_csv("./submission.csv", index=False)